In [1]:
import inspect
from os.path import dirname
from pprint import pprint
from CybORG import CybORG
from CybORG.Simulator.Scenarios.CFileReaderScenarioGenerator import FileReaderScenarioGenerator
from CybORG.reproduction_env import CybORGExtension
from CybORG.Simulator.Host import Status
from CybORG.Shared.Observation import Observation

In [2]:
dic = inspect.getfile(CybORG)
scenario_path = dirname(dic) + f'/Simulator/Scenarios/scenario_files/ThreeWayScenario.yaml'
vulhost_path = dirname(dic) + f'/Simulator/Scenarios/scenario_files/abstractvulnerability/test.yaml'
sg = FileReaderScenarioGenerator(scenario_path)
env = CybORGExtension(sg)
env.load_host_absvul_map_from_yaml(vulhost_path)
env.update_host_absvul_action_space()
from CybORG.Simulator.Actions import *
absvul1 = env.host_absvul_map['API_Gateway']['SB1-CVE-2018-11776']
absvul2 = env.host_absvul_map['ServiceB1']['SB2-CVE-2016-4461']
absvul3 = env.host_absvul_map['ServiceB2']['SB1-CVE-2017-12611']
absvul4 = env.host_absvul_map['ServiceB1']['DB-CVE-2017-12611']
results = env.reset(agent='Red')

In [3]:
# API_Gateway -> Discover ServiceB1
action1 = ExploitLocalVulnerability(session=0,agent='Red',hostname='API_Gateway',absvul=absvul1)
results = env.step(action=action1,agent='Red')
print(f"Observation of 'API_Gateway' action history: {results.observation['action_history']['API_Gateway']['SB1-CVE-2018-11776']}")
print(f"Observation of Discover Sequence: {[f'{k}:{v}' for k,v in results.observation['discovered_sequence'].items() if v!='Unknown']}")
print(f"reward: {results.reward}")
# print("-"*100)
# # ServiceB1 -> Discover ServiceB2
action2 = ExploitRemoteVulnerability(session=0,agent='Red',hostname='ServiceB1',absvul=absvul2)
results = env.step(action=action2,agent="Red")
# pprint(results.observation)
# pprint(results.reward)

Observation of 'API_Gateway' action history: {'success': True, 'failure': False}
Observation of Discover Sequence: ['API_Gateway:0', 'ServiceB1:1']
reward: 5.0


In [9]:
results.observation

{'success': <TrinaryEnum.UNKNOWN: 2>,
 'running_status': {'API_Gateway': 1,
  'ServiceA1': 1,
  'ServiceA2': 1,
  'ServiceB1': 1,
  'ServiceB2': 1,
  'ServiceC1': 1,
  'ServiceC2': 1,
  'DatabaseA': 1,
  'DatabaseB': 2,
  'DatabaseC': 1},
 'suspicous': {'API_Gateway': 1,
  'ServiceA1': 0,
  'ServiceA2': 0,
  'ServiceB1': 0,
  'ServiceB2': 0,
  'ServiceC1': 0,
  'ServiceC2': 0,
  'DatabaseA': 0,
  'DatabaseB': 0,
  'DatabaseC': 0},
 'action_history': {'API_Gateway': {'rollback': {'success': False,
    'failure': False},
   'resotre': {'success': False, 'failure': False}},
  'ServiceA1': {'rollback': {'success': False, 'failure': False},
   'resotre': {'success': False, 'failure': False}},
  'ServiceA2': {'rollback': {'success': False, 'failure': False},
   'resotre': {'success': False, 'failure': False}},
  'ServiceB1': {'rollback': {'success': False, 'failure': False},
   'resotre': {'success': False, 'failure': False}},
  'ServiceB2': {'rollback': {'success': False, 'failure': False},

In [4]:
# Scan through hosts, found two or more suspicious hosts
action = Scan(session=0, agent='Blue')
results = env.step(action=action,agent="Blue")
print(f"scan result: {results.observation['suspicous']}")
print(f"reward: {results.reward}")

scan result: {'API_Gateway': 1, 'ServiceA1': 0, 'ServiceA2': 0, 'ServiceB1': 0, 'ServiceB2': 0, 'ServiceC1': 0, 'ServiceC2': 0, 'DatabaseA': 0, 'DatabaseB': 0, 'DatabaseC': 0}
reward: 1.0


In [5]:
# ServiceB2 -> Exploit ServiceB1(Reverse Session)
action3 = ExploitRemoteVulnerability(session=0,agent='Red',hostname='ServiceB2',absvul=absvul3)
results = env.step(action=action3,agent="Red")
# pprint(results.observation)
# pprint(results.reward)
# PrivilegeEscalate ServiceB1
action4 = PrivilegeEscalate(hostname='ServiceB1',session=0,agent='Red')
results = env.step(action=action4,agent="Red")
# pprint(results.observation)
# pprint(results.reward)
# ServiceB1 -> Exploit DatabaseB(Reverse Session)
action5 = ExploitLocalVulnerability(session=0,agent='Red',hostname='ServiceB1',absvul=absvul4)
results = env.step(action=action5,agent="Red")
# pprint(results.observation)
# pprint(results.reward)
# PrivilegeEscalate DatabaseB
action6 = PrivilegeEscalate(hostname='DatabaseB',session=0,agent='Red')
results = env.step(action=action6,agent="Red")
print(f"Observation of Discover Sequence: {[f'{k}:{v}' for k,v in results.observation['discovered_sequence'].items() if v!='Unknown']}")
print(f"reward: {results.reward}")

Observation of Discover Sequence: ['API_Gateway:0', 'ServiceB1:1', 'ServiceB2:2', 'DatabaseB:3']
reward: 100.0


In [6]:
# Combination of the Blue Actions
# Restore the host with reverse session
action = Restore(session=0,agent='Blue',hostname='DatabaseB')
results = env.step(action=action,agent='Blue')
print(f"status: {results.observation['running_status']['DatabaseB']}")
print(f"reward: {results.reward}")
# # Check if DatabaseB can be exploited again
# action6 = PrivilegeEscalate(hostname='DatabaseB',session=0,agent='Red')
# results = env.step(action=action6,agent="Red")
# pprint(results.observation)
# print("-"*100)
# # rollback the vulnerability so DatabaseB won't be found
# action = RollBackHost(session=0,agent='Blue',hostname='ServiceB1')
# results = env.step(action=action,agent='Blue')
# pprint(results.observation)
# print(f"how many step remain for reimagination:{env.environment_controller.state.hosts['DatabaseB'].reimage_step}")

status: 2
reward: 99.0
